# Imports

In [ ]:
import os
import json
from PIL import Image

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [ ]:
%run ../mimic_cxr.py

In [ ]:
DICOM_DIR = os.path.join(DATASET_DIR.replace('-jpg', ''), 'files')

# Dicom files

In [ ]:
import pydicom as pydcm

In [ ]:
%run ../../utils/dicom.py

In [ ]:
name = 'p19/p19974002/s52374783/76b55d6d-492245e2-c56a31ae-9f0d1cf4-78005e3b.dcm'
# name = 'p19/p19001252/s52265893/431f9ba5-9f643a52-6557b9e3-63ada8e0-c05a6365.dcm'
fpath = os.path.join(DICOM_DIR, name)

In [ ]:
dicom = pydcm.read_file(fpath)
dicom

In [ ]:
dicom['WindowWidth'], dicom['WindowCenter']

In [ ]:
from typing import cast, List

In [ ]:
elem = ds['WindowWidth']
width = cast(List[float], elem.value)[index] if elem.VM > 1 else elem.value
width = cast(float, width)
width

In [ ]:
elem = ds['WindowCenter']
center = (
    cast(List[float], elem.value)[index] if elem.VM > 1 else elem.value
)
center = cast(float, center)
center

In [ ]:
y_min = 0
y_max = 2**ds.BitsStored - 1
y_range = y_max - y_min
arr = arr.astype('float64')

In [ ]:
below = arr <= (center - width / 2)
above = arr > (center + width / 2)
between = np.logical_and(~below, ~above)

In [ ]:
arr[below] = y_min
arr[above] = y_max
if between.any():
    arr[between] = (
        ((arr[between] - center) / width + 0.5) * y_range + y_min
    )

In [ ]:
arr.min(), arr.max(), arr.dtype

In [ ]:
plt.imshow(arr)

In [ ]:
'RescaleSlope' in dicom, 'RescaleIntercept' in dicom

In [ ]:
dicom.BitsStored

In [ ]:
ds = dicom

In [ ]:
arr = dicom.pixel_array
arr.shape, arr.dtype

In [ ]:
arr2 = dicom_to_np(dicom)
arr2.shape, arr.dtype

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.imshow(arr2, cmap='gray')
p(arr2)

plt.subplot(1, 2, 2)
plt.imshow(arr, cmap='gray')
p(arr)

In [ ]:
image = Image.fromarray(arr, mode='I;16')
arr3 = np.array(image, dtype=np.uint16)
image.mode, image.size, arr3.dtype, arr3.shape

In [ ]:
image2 = Image.frombuffer(
    'I;16', (dicom.Columns, dicom.Rows), arr2, 'raw', 'I;16', 0, 1,
)
arr4 = np.array(image2, dtype=np.uint16)
image2.mode, image2.size, arr4.dtype, arr4.shape

In [ ]:
p = lambda a: print(a.min(), a.max(), a.mean(), a.std())
p(arr2)
p(arr3)
p(arr4)

In [ ]:
plt.figure(figsize=(14, 5))

n_rows = 1
n_cols = 3

plt.subplot(n_rows, n_cols, 1)
plt.imshow(arr2, cmap='gray')

plt.subplot(n_rows, n_cols, 2)
plt.imshow(arr3, cmap='gray')

plt.subplot(n_rows, n_cols, 3)
plt.imshow(arr4, cmap='gray')